## Importing dependencies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import sqlalchemy

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

## Importing first dataset

In [2]:
df_path="./Resources/Hospitals.csv"

df_hospitals = pd.read_csv(df_path)

In [3]:
df_hospitals.head()

,organisationid,organisationcode,organisationtype,SubType,sector,OrganisationStatus,IsPimsManaged,organisationname,address1,address2,...,county,postcode,latitude,longitude,ParentODSCode,ParentName,Phone,Email,Website,"Fax,,,"
0,17970,NDA07,Hospital,Hospital,Independent Sector,Visible,True,Walton Community Hospital - Virgin Care Servic...,NaN,Rodney Road,...,Surrey,KT12 3LD,51.379997,-0.406042,NDA,Virgin Care Services Ltd,01932 414205,NaN,NaN,"01932 253674,,,"
1,17981,NDA18,Hospital,Hospital,Independent Sector,Visible,True,Woking Community Hospital (Virgin Care),NaN,Heathside Road,...,Surrey,GU22 7HS,51.315132,-0.556289,NDA,Virgin Care Services Ltd,01483 715911,NaN,NaN,",,,"
2,18102,NLT02,Hospital,Hospital,NHS Sector,Visible,True,North Somerset Community Hospital,North Somerset Community Hospital,Old Street,...,Avon,BS21 6BS,51.437195,-2.847193,NLT,North Somerset Community Partnership Community...,01275 872212,NaN,http://www.nscphealth.co.uk,",,,"
3,18138,NMP01,Hospital,Hospital,Independent Sector,Visible,False,Bridgewater Hospital,120 Princess Road,NaN,...,Greater Manchester,M15 5AT,53.459744,-2.245469,NMP,Bridgewater Hospital (Manchester) Ltd,0161 2270000,NaN,www.bridgewaterhospital.com,",,,"
4,18142,NMV01,Hospital,Hospital,Independent Sector,Visible,True,Kneesworth House,Old North Road,Bassingbourn,...,NaN,SG8 5JP,52.078121,-0.030604,NMV,Partnerships In Care Ltd,01763 255 700,reception_kneesworthhouse@partnershipsincare.c...,www.partnershipsincare.co.uk,",,,"


## Dropping extra columns

In [5]:
clean_df=df_hospitals.drop(["organisationtype", "SubType", "OrganisationStatus", "IsPimsManaged", "ParentODSCode", "Phone", "Email", "Website", "Fax,,,", "ParentName"],axis=1)
clean_df = clean_df.fillna("nan")
clean_df.head()

,organisationid,organisationcode,sector,organisationname,address1,address2,address3,city,county,postcode,latitude,longitude
0,17970,NDA07,Independent Sector,Walton Community Hospital - Virgin Care Servic...,nan,Rodney Road,nan,Walton-on-Thames,Surrey,KT12 3LD,51.379997,-0.406042
1,17981,NDA18,Independent Sector,Woking Community Hospital (Virgin Care),nan,Heathside Road,nan,Woking,Surrey,GU22 7HS,51.315132,-0.556289
2,18102,NLT02,NHS Sector,North Somerset Community Hospital,North Somerset Community Hospital,Old Street,nan,Clevedon,Avon,BS21 6BS,51.437195,-2.847193
3,18138,NMP01,Independent Sector,Bridgewater Hospital,120 Princess Road,nan,nan,Manchester,Greater Manchester,M15 5AT,53.459744,-2.245469
4,18142,NMV01,Independent Sector,Kneesworth House,Old North Road,Bassingbourn,nan,Royston,nan,SG8 5JP,52.078121,-0.030604


In [6]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1209 entries, 0 to 1208
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   organisationid    1209 non-null   int64  
 1   organisationcode  1209 non-null   object 
 2   sector            1209 non-null   object 
 3   organisationname  1209 non-null   object 
 4   address1          1209 non-null   object 
 5   address2          1209 non-null   object 
 6   address3          1209 non-null   object 
 7   city              1209 non-null   object 
 8   county            1209 non-null   object 
 9   postcode          1209 non-null   object 
 10  latitude          1209 non-null   float64
 11  longitude         1209 non-null   float64
dtypes: float64(2), int64(1), object(9)
memory usage: 113.5+ KB


In [8]:
clean_df1 = clean_df.astype({'latitude':'float' , 'longitude':'float' }) 

clean_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1209 entries, 0 to 1208
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   organisationid    1209 non-null   int64  
 1   organisationcode  1209 non-null   object 
 2   sector            1209 non-null   object 
 3   organisationname  1209 non-null   object 
 4   address1          1209 non-null   object 
 5   address2          1209 non-null   object 
 6   address3          1209 non-null   object 
 7   city              1209 non-null   object 
 8   county            1209 non-null   object 
 9   postcode          1209 non-null   object 
 10  latitude          1209 non-null   float64
 11  longitude         1209 non-null   float64
dtypes: float64(2), int64(1), object(9)
memory usage: 113.5+ KB


In [9]:
#connection string to load data to sequel
protocol = "postgresql"
username = "postgres"
host = "localhost"
port = 5432
database_name = "project3"
Password ="root"
connection_string = f"{protocol}://{username}:{Password}@{host}:{port}/{database_name}"
engine = create_engine(connection_string)        
engine.table_names()

C:\Users\prern\AppData\Local\Temp\ipykernel_18900\2373570809.py:10: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['hospitals']

In [10]:
clean_df1.to_sql('hospitals',engine, if_exists='append', index=False)

209

## Creating list of coordinates for cluster map

In [10]:
cord = [{"latitude": lat, "longitude": lon, "oname":oname,"county":county,"city":city} for lat, lon,oname,county,city in zip(clean_df1['latitude'], clean_df1 ['longitude'], clean_df1 ['organisationname'], clean_df1 ['county'], clean_df1 ['city'])]

KeyError: 'latitude'

In [ ]:
cord